Env: in py, no spark.

Time: 20 sec to run all.

# Data

In [130]:
import numpy as np
import pandas as pd 
import sys
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import time

# change this to your own path
data_origin_df = pd.read_csv("/mnt/nfsMountPoint/datasets/gutenberg_data/data-23k-utf.csv")

In [131]:
data_origin_df.head()

,filename,book_name,author,total_words,avg_sentence_size,!,#,'',%,$,...,},~,neg,neu,pos,compound,ID,smallClusterId,mediumClusterId,largeClusterId
0,18241.txt,"Tea-Cup Reading, and the Art of Fortune-Tellin...",'A Highland Seer',12096,14.573494,0.000412,0.000412,0.020576,0.000412,0.000823,...,0.002881,0.005761,0.033866,0.892546,0.073593,0.160735,18241,1.0,12,155.0
1,33342.txt,Old New Zealand:,'A Pakeha Maori' [Frederick Edwa [Maning],49598,22.909007,0.020902,0.000085,0.079716,0.000085,0.000169,...,0.000000,0.000000,0.072623,0.847508,0.079876,0.027016,33342,35.0,99,265.0
2,19238.txt,Foundations of World Unity,'Abdu'l-Baha,44254,18.807480,0.009432,0.000175,0.024803,0.000175,0.000349,...,0.009956,0.019913,0.055716,0.820704,0.123586,0.192782,19238,35.0,99,319.0
3,19284.txt,Paris Talks,'Abdu'l-Baha,42512,21.667686,0.045009,0.000156,0.004361,0.000156,0.000311,...,0.008877,0.017754,0.066816,0.783651,0.149555,0.270730,19284,35.0,99,319.0
4,19237.txt,The Secret of Divine Civilization,'Abdu'l-Baha,30276,25.876923,0.007941,0.000199,0.054199,0.000199,0.000397,...,0.011316,0.022633,0.057088,0.834483,0.108442,0.213498,19237,35.0,99,361.0


In [132]:
try:
    usr_title = 'Philosophical Transactions of the Royal Society - Vol 1 - 1666'
    usr_author = 'Various'
    found = data_origin_df[(data_origin_df.book_name == usr_title) & (data_origin_df.author == usr_author)]
    found_book = found['ID']
    id_target = found_book.tolist()[0]
except IndexError:
    print("**** Title and/or Author Incorrect or Book is not in corpus****\nPlease try again")
    # sys.exit(1)
else:
    print("************** Searching for Book *******************")
    #break

************** Searching for Book *******************


# Drop Symbols

In [133]:
# status = "**************** Adjusting Data *******************"

data = data_origin_df

# dropping one-side symbol columnss
data = data.drop('(', 1)
data = data.drop('[', 1)
data = data.drop('{', 1)
data = data.drop('>', 1)
# dropping other columnss
data = data.drop('#', 1)
data = data.drop('&', 1)
data = data.drop('/', 1)
data = data.drop('\\', 1)
data = data.drop('*', 1)
data = data.drop('@', 1)
data = data.drop('_', 1)
data = data.drop('^', 1)

data_dropped_symbols_df = data.copy()

In [134]:
data_dropped_symbols_df.shape

(23789, 31)

In [135]:
data_dropped_symbols_df.head()

,filename,book_name,author,total_words,avg_sentence_size,!,'',%,$,')',...,},~,neg,neu,pos,compound,ID,smallClusterId,mediumClusterId,largeClusterId
0,18241.txt,"Tea-Cup Reading, and the Art of Fortune-Tellin...",'A Highland Seer',12096,14.573494,0.000412,0.020576,0.000412,0.000823,0.015226,...,0.002881,0.005761,0.033866,0.892546,0.073593,0.160735,18241,1.0,12,155.0
1,33342.txt,Old New Zealand:,'A Pakeha Maori' [Frederick Edwa [Maning],49598,22.909007,0.020902,0.079716,0.000085,0.000169,0.009901,...,0.000000,0.000000,0.072623,0.847508,0.079876,0.027016,33342,35.0,99,265.0
2,19238.txt,Foundations of World Unity,'Abdu'l-Baha,44254,18.807480,0.009432,0.024803,0.000175,0.000349,0.006638,...,0.009956,0.019913,0.055716,0.820704,0.123586,0.192782,19238,35.0,99,319.0
3,19284.txt,Paris Talks,'Abdu'l-Baha,42512,21.667686,0.045009,0.004361,0.000156,0.000311,0.009500,...,0.008877,0.017754,0.066816,0.783651,0.149555,0.270730,19284,35.0,99,319.0
4,19237.txt,The Secret of Divine Civilization,'Abdu'l-Baha,30276,25.876923,0.007941,0.054199,0.000199,0.000397,0.023824,...,0.011316,0.022633,0.057088,0.834483,0.108442,0.213498,19237,35.0,99,361.0


# Add Dummy Variables of Cluster Indicesand Authors

In [136]:
data = data_dropped_symbols_df

# data = pd.concat([data, pd.get_dummies(data['Cluster'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['smallClusterId'], prefix="general")], axis=1)  #and for each cluster
data = pd.concat([data, pd.get_dummies(data['mediumClusterId'], prefix="medium")], axis=1) 
data = pd.concat([data, pd.get_dummies(data['largeClusterId'], prefix="specific")], axis=1) 
data = pd.concat([data, pd.get_dummies(data['author'])], axis=1)  #now you have a binary variable for each author

data_concat_dummy_df = data.copy()

In [137]:
data_concat_dummy_df.shape

(23789, 10490)

In [138]:
data_concat_dummy_df.columns[713:]

Index(['specific_477.0', 'specific_478.0', 'specific_479.0', '#NAME?',
       ''A Highland Seer'', ''A Pakeha Maori' [Frederick Edwa [Maning]',
       ''Abdu'l-Baha', '(A.E.) George William Russell',
       '(A.E.)George William Russell', '(AKA Charlotte Bronte) Currer Bell',
       ...
       '[AKA Ohiyesa], Charles A. Eastman',
       '[Titus Lucretius Carus] Lucretius', 'anonymous',
       'by (AKA B. M. Sinclair) B. M. Bower', 'by Marshall Saunders',
       'by Robert Barr', 'the Bab', 'unknown', 'various',
       '姉_mile Gigault de La B抇dolli挔re'],
      dtype='object', length=9777)

In [139]:
data_concat_dummy_df.head()

,filename,book_name,author,total_words,avg_sentence_size,!,'',%,$,')',...,"[AKA Ohiyesa], Charles A. Eastman",[Titus Lucretius Carus] Lucretius,anonymous,by (AKA B. M. Sinclair) B. M. Bower,by Marshall Saunders,by Robert Barr,the Bab,unknown,various,姉_mile Gigault de La B抇dolli挔re
0,18241.txt,"Tea-Cup Reading, and the Art of Fortune-Tellin...",'A Highland Seer',12096,14.573494,0.000412,0.020576,0.000412,0.000823,0.015226,...,0,0,0,0,0,0,0,0,0,0
1,33342.txt,Old New Zealand:,'A Pakeha Maori' [Frederick Edwa [Maning],49598,22.909007,0.020902,0.079716,0.000085,0.000169,0.009901,...,0,0,0,0,0,0,0,0,0,0
2,19238.txt,Foundations of World Unity,'Abdu'l-Baha,44254,18.807480,0.009432,0.024803,0.000175,0.000349,0.006638,...,0,0,0,0,0,0,0,0,0,0
3,19284.txt,Paris Talks,'Abdu'l-Baha,42512,21.667686,0.045009,0.004361,0.000156,0.000311,0.009500,...,0,0,0,0,0,0,0,0,0,0
4,19237.txt,The Secret of Divine Civilization,'Abdu'l-Baha,30276,25.876923,0.007941,0.054199,0.000199,0.000397,0.023824,...,0,0,0,0,0,0,0,0,0,0


# Drop Meta Data for Scaling

In [140]:
data = data_concat_dummy_df

titles = data['book_name']
authors = data['author']
IDs = data['ID']

data = data.drop('ID', 1)
data = data.drop('book_name', 1)
data = data.drop('mediumClusterId', 1)
data = data.drop('smallClusterId', 1)
data = data.drop('largeClusterId', 1)
data = data.drop('author', 1)
data = data.drop('filename', 1)
data = data.fillna(0)

data = pd.DataFrame(MinMaxScaler().fit_transform(data), columns=data.columns)
# data.to_csv("Normalized output.csv")
data['ID'] = IDs  #had to remove this earlier because we didn't want IDs to be scaled with the other columns

data_dropped_meta_df = data.copy()

In [141]:
data_dropped_meta_df.shape

(23789, 10484)

In [142]:
data_dropped_meta_df.head()

,total_words,avg_sentence_size,!,'',%,$,')',+,-,",",...,[Titus Lucretius Carus] Lucretius,anonymous,by (AKA B. M. Sinclair) B. M. Bower,by Marshall Saunders,by Robert Barr,the Bab,unknown,various,姉_mile Gigault de La B抇dolli挔re,ID
0,0.003294,0.033280,0.002592,0.053023,0.001682,0.005794,0.095543,0.000000,0.186239,0.484633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18241
1,0.016726,0.053029,0.132264,0.206189,0.000346,0.001191,0.062127,0.000000,0.176014,0.519594,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33342
2,0.014812,0.043311,0.059679,0.063971,0.000714,0.002459,0.041650,0.000000,0.067236,0.475837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19238
3,0.014188,0.050088,0.284819,0.011027,0.000636,0.002193,0.059612,0.000000,0.086962,0.601692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19284
4,0.009805,0.060061,0.050243,0.140103,0.000811,0.002795,0.149490,0.000939,0.080141,0.605413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19237


# Drop ID Before KNN

In [143]:
data = data_dropped_meta_df

input_query = data[data['ID']==id_target]
input_query = input_query.drop('ID', 1)

data = data.drop('ID', 1)

data_no_id_before_knn_df = data.copy()

In [144]:
data_no_id_before_knn_df.shape

(23789, 10483)

In [145]:
data_no_id_before_knn_df.head()

,total_words,avg_sentence_size,!,'',%,$,')',+,-,",",...,"[AKA Ohiyesa], Charles A. Eastman",[Titus Lucretius Carus] Lucretius,anonymous,by (AKA B. M. Sinclair) B. M. Bower,by Marshall Saunders,by Robert Barr,the Bab,unknown,various,姉_mile Gigault de La B抇dolli挔re
0,0.003294,0.033280,0.002592,0.053023,0.001682,0.005794,0.095543,0.000000,0.186239,0.484633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.016726,0.053029,0.132264,0.206189,0.000346,0.001191,0.062127,0.000000,0.176014,0.519594,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.014812,0.043311,0.059679,0.063971,0.000714,0.002459,0.041650,0.000000,0.067236,0.475837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.014188,0.050088,0.284819,0.011027,0.000636,0.002193,0.059612,0.000000,0.086962,0.601692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.009805,0.060061,0.050243,0.140103,0.000811,0.002795,0.149490,0.000939,0.080141,0.605413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# KNN Train/Fit

In [146]:
data = data_no_id_before_knn_df

# 10 sec
knnModel = NearestNeighbors().fit(data)

# KNN Predict/Match

In [147]:
output = knnModel.kneighbors(input_query, 15) # it is using pairwise euclidean distance

distance_listing = output[0][0]
nn_index_listing = output[1][0]

In [148]:
output[0]

array([[ 0.        ,  0.32161454,  0.42659012,  0.45384219,  0.49641305,
         1.43799242,  1.43885648,  1.44033766,  1.44182637,  1.44297306,
         1.44371745,  1.44537928,  1.44545209,  1.44557848,  1.4457683 ]])

In [149]:
data = data_no_id_before_knn_df

fulldata = data
fulldata['author'] = authors
fulldata['book_name'] = titles

full_results = fulldata.iloc[nn_index_listing, :]
results = full_results[['book_name','author']]

status = "************* Recommendations ***************"
print(status)
print(results)

************* Recommendations ***************
                                               book_name  \
20639  Philosophical Transactions of the Royal Societ...   
20640  Encyclopaedia Britannica, 11th Edition, Volume...   
20638  The Quarterly Review, Volume 162, No. 324, Apr...   
20637      The Great Events by Famous Historians, Vol. 1   
20641                            The Library and Society   
2159            A Budget of Paradoxes, Volume II (of II)   
11194                          Roumania Past and Present   
23561                            English Book Collectors   
20700        The New Gresham Encyclopedia. Vol. 1 Part 1   
617                                           Fine Books   
12711                           Shakespearean Playhouses   
20701        The New Gresham Encyclopedia. Vol. 1 Part 3   
11962               Historical Sketches, Volume I (of 3)   
8080        A History of Art in Chaldaea & Assyria, v. 1   
11476  Games for the Playground, Home, School and Gym.

In [150]:
print(full_results.columns)

Index(['total_words', 'avg_sentence_size', '!', '''', '%', '$', '')'', '+',
       '-', ',',
       ...
       'anonymous', 'by (AKA B. M. Sinclair) B. M. Bower',
       'by Marshall Saunders', 'by Robert Barr', 'the Bab', 'unknown',
       'various', '姉_mile Gigault de La B抇dolli挔re', 'author', 'book_name'],
      dtype='object', length=10485)
